# Library Downloads

In [ ]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score
!pip install -q trl
!pip install -q accelerate
!pip install -q optimum langchain langchain-community
!pip install accelerate
print()

# Imports

In [1]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

2024-06-07 10:14:35.097231: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-07 10:14:35.097293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-07 10:14:35.098733: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ·····································
Add token as git credential? (Y/n)  n


Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Data Cleaning
- Filling NaNs with empty strings
- merging 2 datasets based on parent_asin
- merging product categories into a single string

In [4]:
product_asin = pd.read_csv("/kaggle/input/re-shl/product_asin.csv")
reviews_supplements = pd.read_csv("/kaggle/input/re-shl/reviews_supplements.csv")
product_asin = product_asin.fillna("")
reviews_supplements = reviews_supplements.fillna("")
reviews_supplements["text"] = reviews_supplements["text"].apply(lambda x: '\n\n previous user review: '+ x )
combined_data = reviews_supplements.groupby("parent_asin")["text"].apply(lambda x: ("").join(x))

In [5]:
def concatenate_non_empty(row):
    return ','.join([val for val in row if val != ''])

product_asin['concatenated'] = product_asin.loc[:, ["cat1",	"cat2","cat3","cat4","cat5","cat6"]].apply(concatenate_non_empty, axis=1)

In [6]:
product_asin = product_asin.loc[:, ["title", "parent_asin", "concatenated"]]
result_dict = product_asin.set_index('title').T.apply(list).to_dict()

/tmp/ipykernel_143/1949421406.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = product_asin.set_index('title').T.apply(list).to_dict()


In [7]:
combined_data = pd.DataFrame(combined_data)
merged_df = pd.merge(product_asin, combined_data, on='parent_asin', how='right')

In [8]:
merged_df.head()

,title,parent_asin,concatenated,text
0,"BComplex 50 – Supports Energy Production, Nerv...",B00012ND5G,'Heart Health Event'],\n\n previous user review: I bought this along...
1,Twinlab Ultra GLA 300 -- 60 Softgels,B00013Z0ZQ,'Heart Health Event'],\n\n previous user review: great product\n\n p...
2,NonOily Dry A,B00013Z1KA,'Heart Health Event'],\n\n previous user review: Excellent Product ....
3,Vitamin A D,B00014D0IY,'Heart Health Event'],\n\n previous user review: I have been taking ...
4,Country Life Liver Support Factors -- 100 Tablets,B00014DY62,'Heart Health Event'],\n\n previous user review: I have had good res...


# Model Preparations
- Quantization
- model loading and making parameter pipeline

In [9]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [10]:
# def gen(model,p, maxlen=100, sample=True):
#     toks = eval_tokenizer(p, return_tensors="pt")
#     res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.1,num_beams=1,top_p=0.95,).to('cpu')
#     return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

In [25]:
MODEL_NAME = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True,device_map="auto",
                                                      quantization_config=bnb_config,
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 128
generation_config.temperature = 0.60
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.00

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Synthetic Data Generation
- generating bert embeddings of generated and GT reviews
- generating llama outputs
- calculating cosine similarity

In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

def get_bert_embedding(text):
    """
    Generates the BERT embedding for a given text.

    Args:
    text (str): The input text to be embedded.

    Returns:
    torch.Tensor: The mean of the last hidden state from the BERT model, representing the text embedding.
    """
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    return outputs.last_hidden_state.mean(dim=1)

def calculate_similarity(text1, text2):
    """
    Calculates the cosine similarity between the BERT embeddings of two texts.

    Args:
    text1 (str): The first input text.
    text2 (str): The second input text.

    Returns:
    float: The cosine similarity score between the embeddings of the two texts.
    """
    embedding1 = get_bert_embedding(text1)
    
    embedding2 = get_bert_embedding(text2)
    
    return cosine_similarity(embedding1.cpu().numpy(), embedding2.cpu().numpy())[0][0]


final_llama_results = {}
n_reviews = 2
for idx in range(5):
    llama_output = set()
    name = merged_df.iloc[idx, 0]
    text = merged_df.iloc[idx, -1]
    user_reviews = text.split("\n\n")[1:]
    keywords = merged_df.iloc[idx, -2]
    counter = 0

    while (len(llama_output) < n_reviews) and (counter < 15):
        result = llm(
            f"product domain: {keywords}, product name: {name} \n\n Use these samples and enerate similar review. Use information from given texts. Keep review length similar to samples. {text}. \n\n"
        )
        new_reviews = list(set(result.split("\n\n")[1:][len(user_reviews):]))
        counter += 1
        for review in new_reviews:
            if review in user_reviews:
                continue
            else:
                if len(llama_output) >= n_reviews:
                    break
                for user_review in user_reviews:
                    similarity = calculate_similarity(review, user_review)
                    if (similarity > 0.75) and (similarity < 0.90):
                        llama_output.add(review)
                    if len(llama_output) == n_reviews:
                        break

    llama_output = list(llama_output)[:2]
    final_llama_results[name] = llama_output

print(final_llama_results)


{'BComplex 50 – Supports Energy Production, Nervous System Function Nutrient Metabolism – Excellent Source of B1, B2, B6, B12, Niacin, Folic Acid Biotin (100 Softgels) by The Vitamin Shoppe': [" previous user review: I'm giving it a 5 star because I'm hoping it works. I have a B12 deficiency and was told to take B-12 shots. I'm hoping this will help. I'm also hoping it won't hurt me since I have high blood pressure. ", ' previous user review: I had a blood test recently, and found out I was deficient in vitamin B12. I was also told that I am not absorbing it properly. So I got this vitamin, and it helps me. I take it every day,'], 'Twinlab Ultra GLA 300 -- 60 Softgels': [" previous user review: I've used this product for a while and I've seen great results. ", ' previous user review: Came as expectedly. '], 'NonOily Dry A': [' previous user review: El producto es excelente, me gusta mucho. ', ' previous user review: No me ha funcionado, eso de que "es la mejor" no es verdad.'], 'Vitami

In [27]:
for i in final_llama_results:
    print(i, len(final_llama_results[i]))

BComplex 50 – Supports Energy Production, Nervous System Function Nutrient Metabolism – Excellent Source of B1, B2, B6, B12, Niacin, Folic Acid Biotin (100 Softgels) by The Vitamin Shoppe 2
Twinlab Ultra GLA 300 -- 60 Softgels 2
NonOily Dry A 2
Vitamin A D 2
Country Life Liver Support Factors -- 100 Tablets 2


# Processing and storage

In [28]:
col1 = []
col2 = []
col3 = []
for i in final_llama_results:
    col1.append(i)
    col2.append(final_llama_results[i][0])
    col3.append(final_llama_results[i][1])
    
final_llama = {"name" : col1, "rev1" : col2, "rev2": col3}
# final_llama_results = pd.DataFrame(final_llama_results)

In [30]:
pd.DataFrame(final_llama).to_csv("synth_reviews.csv")

In [29]:
pd.DataFrame(final_llama)

,name,rev1,rev2
0,"BComplex 50 – Supports Energy Production, Nerv...",previous user review: I'm giving it a 5 star ...,previous user review: I had a blood test rece...
1,Twinlab Ultra GLA 300 -- 60 Softgels,previous user review: I've used this product ...,previous user review: Came as expectedly.
2,NonOily Dry A,previous user review: El producto es excelent...,"previous user review: No me ha funcionado, es..."
3,Vitamin A D,previous user review: I have been taking this...,1. I have been taking this product for years....
4,Country Life Liver Support Factors -- 100 Tablets,previous user review: Very good value!.,previous user review:
